# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [13]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [14]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [15]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [16]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [17]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [18]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [19]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [20]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [21]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [22]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [23]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [24]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [25]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for our engagement strategy.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_company_background.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './instructions/DeepLearningAI_company_background

Action Input: {"search_query": "DeepLearningAI key personnel and recent milestones"} 


Search results: Title: The Batch | DeepLearning.AI | AI News & Insights
Link: https://www.deeplearning.ai/the-batch/
Snippet: The Batch AI News and Insights: Recently Meta made headlines with unprecedented, massive compensation packages for AI model builders exceeding $100M (sometimes ...
---
Title: Top AI Stories of 2024! Agents Rise, Prices Fall, Models ...
Link: https://www.deeplearning.ai/the-batch/issue-281/
Snippet: Shazeer joined Google's Deep Learning research team, and other new hires set to work on Google's chat services.
---
Title: DeepLearning.AI Management Team
Link: https://www.cbinsights.com/company/deeplearningai/people
Snippet: Explore {DeepLearning.AI's key management people. Discover current leadership team members including founders, CEO, other executives and board directors.
---
Title: How DeepLearning.AI created quality courses
Link: https://www.linkedin.com/posts/andrewyng_ope

- Display the final result as Markdown.

In [26]:
from IPython.display import Markdown
Markdown(result)

Email Draft 1:

Subject: Congratulations on DeepLearningAI's Recent Achievements!

Dear Andrew Ng,

I hope this message finds you well. I wanted to take a moment to congratulate DeepLearningAI on the recent dramatic advances in AI in 2024. The agentic systems' improved abilities to reason, use tools, and control desktop applications are truly remarkable milestones that showcase your commitment to innovation and excellence.

At CrewAI, we share your passion for pushing the boundaries of AI education and providing world-class learning experiences to a global audience. Our solutions are designed to complement DeepLearningAI's existing programs by offering additional tools, resources, and training materials to enhance the learning journey for over 7 million learners worldwide.

I believe that a collaboration between our two organizations could lead to even greater achievements in the field of AI education. By combining our expertise and resources, we can create joint educational content and workshops that will further empower learners and drive innovation in the industry.

I look forward to the possibility of working together and contributing to the continued success of DeepLearningAI. Please let me know if you would be interested in exploring partnership opportunities or discussing how our solutions can support your goals.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

Email Draft 2:

Subject: Enhancing DeepLearningAI's Educational Offerings

Dear Andrew Ng,

I recently came across the Generative AI for Everyone course offered by DeepLearningAI, and I was truly impressed by the pioneering work you are doing in the field of AI education. Your commitment to providing expert-led educational experiences to millions of learners is inspiring, and I believe that our solutions at CrewAI can further enhance the learning experience for your global community of AI talent.

Our personalized training sessions and workshops are designed to empower learners with the skills and knowledge they need to succeed in the ever-evolving world of AI. By collaborating with DeepLearningAI, we can create guest lectures and provide expert insights that will deepen the understanding of AI concepts and technologies.

I am excited about the opportunity to work together and explore how our solutions can support your mission of democratizing AI education. Please let me know if you would be interested in discussing partnership opportunities or learning more about how CrewAI can contribute to DeepLearningAI's success.

Thank you for your time and consideration.

Best regards,

[Your Name]
Lead Sales Representative
CrewAI